In [1]:
import sys
import folium
import geopandas as gpd

# path to folder to CityGeoTools
folder = "/var/essdata/IDU/other/Rest_/refactored/CityGeoTools"
sys.path.append(folder)

from metrics.data import CityInformationModel as BaseModel

/var/essdata/IDU/venvs/common_venv/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### SpaceMatrix

The metric is designed to determine the type of urban area by a combination of indices: 
- Floor Space Index (FSI) - the ratio of the area of all floors of buildings and structures to the area of the block; 
- Ground Space Index (GSI) - the ratio of the area occupied by buildings and structures to the area of the block; 
- the average number of storeys of buildings in the quarter (L); 
- open space index (OSR) - the ratio of undeveloped space at ground level to the area of all floors of buildings and structures in the quarter; 
- Mixed use index (MXI) is defined as the ratio of the living area of all floors of buildings to the area of all floors of buildings and structures. 

To call the metric, first create your own City Information Model (the way how it is shown [here](https://github.com/iduprojects/CityGeoTools/blob/master/notebook_examples/create_city_information_model.ipynb)) and upload the [following files](https://github.com/iduprojects/CityGeoTools/tree/master/data_specification/visibility_analysis) into it:
- geojson file with urban buildings;
- geojson file with urban blocks.

In [2]:
city_model = BaseModel.CityInformationModel(city_name="Saint-Petersburg", city_crs=32636, cwd="../")
city_model.update_layer("Buildings", "./data/buildings.geojson")
city_model.update_layer("Blocks", "./data/blocks.geojson")

Validation of Buildings layer...
Buildings layer loaded successfully!
Validation of Blocks layer...
Blocks layer loaded successfully!


In [3]:
from metrics.calculations import spacematrix
blocks_morphotypes = spacematrix.Spacematrix(city_model).get_morphotypes()
blocks_morphotypes = gpd.GeoDataFrame.from_features(blocks_morphotypes).set_crs(4326)

/var/essdata/IDU/venvs/common_venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [14]:
def style_fn(feature):
    colors = [
        'red', 'blue', 'green', 'orange', 'purple', 'pink', 'cadetblue', 'darkblue', 
        'lightgreen', 'darkred', 'yellow'
    ]
    return {'fillColor': colors[int(feature["properties"]["spacematrix_cluster"])]}

blocks_morphotypes_61 = blocks_morphotypes[blocks_morphotypes["administrative_unit_id"] == 61]
blocks_morphotypes_61 = blocks_morphotypes_61.dropna(subset=["spacematrix_cluster"])
map = folium.Map(location = [59.931, 30.250], zoom_start = 13, tiles='CartoDB positron')
geo_j = folium.GeoJson(
    data=blocks_morphotypes_61, 
    style_function=style_fn,
    tooltip=folium.features.GeoJsonTooltip(["FSI", "GSI", "L", "MXI", "OSR", "spacematrix_cluster"])).add_to(map)
geo_j.add_to(map)
map